In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from d2d import *


filename = "/data/fast0/datasets/Rhone_data_continuous.h5"
f = h5py.File(filename, 'r')
print("Keys: %s" % f.keys())

Keys: <KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>


In [2]:
# # Put data in a data frame
# das = np.transpose([chan[750] for chan in f['DAS Data']])

# # Might want to try this later:
# # das = np.log10(np.transpose([chan[750] for chan in f['DAS Data']]))

# discharge = np.array(f['Discharge'])
# df = pd.DataFrame(data={'das':das,'discharge':discharge})

In [3]:
das_data_all = f['DAS Data'][:]
discharge = f['Discharge'][:]

df_all_chan = pd.DataFrame(das_data_all)
df_all_chan['Discharge'] = discharge
df_all_chan.head()

,0,1,2,3,4,5,6,7,8,9,...,2487,2488,2489,2490,2491,2492,2493,2494,2495,Discharge
0,1737.318436,1932.321258,1788.269101,3706.977556,1831.756626,1768.674227,1908.850695,3690.341997,1804.145150,2437.436803,...,3603.932847,2066.070800,2215.669390,1881.772352,3990.883595,1584.965970,1787.281257,1862.912440,4160.545583,9.947179
1,8161.349279,8245.783723,8118.189935,528.834231,468.890511,714.714310,728.051775,158.440806,82.719811,116.949415,...,8238.452283,8204.901437,8194.671705,8228.769841,8211.066134,8229.370787,8218.436955,8210.143402,8178.032001,9.934657
2,8242.942236,8205.640252,8093.980026,510.832905,451.884623,709.510655,709.932307,153.715844,83.404382,116.112236,...,8184.500407,8221.490571,8202.429647,8194.927561,8197.675324,8188.718827,8189.658071,8225.017259,8163.918322,9.929290
3,8159.780047,8196.654224,8113.873839,489.734947,438.657688,690.418208,701.896139,158.058233,83.003254,112.787627,...,8173.860868,8156.187127,8151.113841,8235.843141,8145.360321,8192.392266,8169.275389,8129.926491,8161.652382,9.923923
4,8198.498465,8161.470921,8120.452338,521.425978,444.272741,689.710217,695.164917,160.246640,84.054029,111.125592,...,8198.827286,8234.815465,8219.987333,8140.961605,8193.517740,8157.154485,8241.648596,8229.194191,8234.212679,9.918557


In [4]:
# Plot the raw data

# fig,ax=plt.subplots(figsize=(15,5))
# ax.plot(das)
# ax.set_ylabel('DAS-measured strain rate')
# ax2=plt.twinx()
# ax2.plot(discharge,'-r')
# ax2.set_ylabel('Discharge')

In [5]:
# Split the data
column_indices = {name: i for i, name in enumerate(df_all_chan.columns)}
n = len(df_all_chan)

train_df = df_all_chan[int(n*0.3):int(n*1.0)]
val_df = df_all_chan[int(n*0.0):int(n*0.2)]
test_df = df_all_chan[int(n*0.2):int(n*0.3)]

In [6]:
input_columns = list(np.arange(0,2308,1))

In [7]:
# Normalize
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [8]:
# # Visualize the normalized data
# df_std = (df_all_chan - train_mean) / train_std
# df_std = df_std.melt(var_name='Column', value_name='Normalized')
# plt.figure(figsize=(12, 6))
# ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
# _ = ax.set_xticklabels(df.keys(), rotation=90)

# Does not like to visulaize with all the channels applied

In [9]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=0,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Discharge'],
    input_columns=input_columns)

# Defining the models

In [10]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(1)
])

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

dnn_model = tf.keras.models.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

In [11]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    return history

In [12]:
history = compile_and_fit(linear, single_step_window)

val_performance={}
performance={}
val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

Epoch 1/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.6119 - mean_absolute_error: 0.5204 - val_loss: 1.0371 - val_mean_absolute_error: 0.8768
Epoch 2/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.2783 - mean_absolute_error: 0.3703 - val_loss: 0.8092 - val_mean_absolute_error: 0.7715
Epoch 3/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.2851 - mean_absolute_error: 0.3546 - val_loss: 0.6132 - val_mean_absolute_error: 0.6698
Epoch 4/20
1373/1373 [==============================] - 9s 7ms/step - loss: 0.4054 - mean_absolute_error: 0.3823 - val_loss: 0.7791 - val_mean_absolute_error: 0.7629
Epoch 5/20
393/393 [==============================] - 7s 538us/step - loss: 0.7211 - mean_absolute_error: 0.7355


In [15]:
# inputs, labels = single_step_window.example

# predictions = np.array(linear(inputs)).flatten()
# inputs = np.array(inputs).flatten()
# labels = np.array(labels).flatten()

# plt.subplots()
# plt.plot(labels,predictions,'ok')
# plt.plot((-1,1),(-1,1))
# plt.xlabel('Actual Normalized Discharge')
# plt.ylabel('Predicted Normalized Discharge')

# plt.subplots()
# plt.plot(inputs, labels,'ok')
# # for xx, yy in single_step_window.val.take(5):
# #     plt.plot(np.array(xx[:,0,:]).flatten(),np.array(yy).flatten(),'or')
# plt.xlabel('Actual Normalized DAS')
# plt.ylabel('Normalized Discharge (Actual and Modeled)')
# weights=linear.layers[0].kernel[:,0].numpy()
# biases=linear.layers[0].bias.numpy()
# plt.plot((-1,3.0),weights[0]*np.array((-1,3.0))+biases[0])

# # Compare to linear least squares
# from scipy import stats
# slope, intercept, r_value, p_value, std_err = stats.linregress(inputs,labels)
# plt.plot((-1,3.0),slope*np.array((-1,3.0))+intercept)

In [16]:
multi_step_window = WindowGenerator(
    input_width=100, label_width=1, shift=0,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Discharge'],
    input_columns=input_columns)
multi_step_window

Total window size: 100
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Input column name(s): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148

# Multistep Model Runs

In [17]:
history = compile_and_fit(linear, multi_step_window)

val_performance['Multistep_Linear'] = linear.evaluate(multi_step_window.val)
performance['Multistep_Linear'] = linear.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.1344 - mean_absolute_error: 0.2695 - val_loss: 0.7284 - val_mean_absolute_error: 0.7384
Epoch 2/20
1370/1370 [==============================] - 24s 17ms/step - loss: 0.1251 - mean_absolute_error: 0.2594 - val_loss: 0.6269 - val_mean_absolute_error: 0.6758
Epoch 3/20
1370/1370 [==============================] - 24s 17ms/step - loss: 0.1176 - mean_absolute_error: 0.2517 - val_loss: 0.7432 - val_mean_absolute_error: 0.7373
Epoch 4/20
390/390 [==============================] - 10s 10ms/step - loss: 0.7433 - mean_absolute_error: 0.7431


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277]}

In [18]:
history = compile_and_fit(lstm_model, multi_step_window)

val_performance['Multistep_LSTM'] = lstm_model.evaluate(multi_step_window.val)
performance['Multistep_LSTM'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 75s 54ms/step - loss: 0.0535 - mean_absolute_error: 0.1463 - val_loss: 0.7692 - val_mean_absolute_error: 0.6912
Epoch 2/20
1370/1370 [==============================] - 74s 54ms/step - loss: 0.0197 - mean_absolute_error: 0.0953 - val_loss: 1.0103 - val_mean_absolute_error: 0.7952
Epoch 3/20
390/390 [==============================] - 15s 21ms/step - loss: 1.0369 - mean_absolute_error: 0.8265


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235]}

In [19]:
history = compile_and_fit(dnn_model, multi_step_window)

val_performance['Multistep_DNN'] = dnn_model.evaluate(multi_step_window.val)
performance['Multistep_DNN'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 26s 19ms/step - loss: 0.0762 - mean_absolute_error: 0.1825 - val_loss: 0.8696 - val_mean_absolute_error: 0.7965
Epoch 2/20
1370/1370 [==============================] - 26s 19ms/step - loss: 0.0206 - mean_absolute_error: 0.1074 - val_loss: 0.7964 - val_mean_absolute_error: 0.7529
Epoch 3/20
1370/1370 [==============================] - 24s 17ms/step - loss: 0.0436 - mean_absolute_error: 0.1330 - val_loss: 0.8491 - val_mean_absolute_error: 0.7752
Epoch 4/20
1370/1370 [==============================] - 24s 18ms/step - loss: 0.0135 - mean_absolute_error: 0.0876 - val_loss: 0.7290 - val_mean_absolute_error: 0.7033
Epoch 5/20
1370/1370 [==============================] - 24s 17ms/step - loss: 0.0123 - mean_absolute_error: 0.0841 - val_loss: 0.7599 - val_mean_absolute_error: 0.7038
Epoch 6/20
390/390 [==============================] - 11s 10ms/step - loss: 0.8214 - mean_absolute_error: 0.7626


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046]}

# Retraining same model on different partitions of training, val, and test

In [20]:
# # Split the data
# column_indices = {name: i for i, name in enumerate(df_all_chan.columns)}
# n = len(df_all_chan)

train_df = df_all_chan[int(n*0.0):int(n*0.7)]
val_df = df_all_chan[int(n*0.8):int(n*1.0)]
test_df = df_all_chan[int(n*0.7):int(n*0.8)]

In [21]:
history = compile_and_fit(lstm_model, multi_step_window)

val_performance['Multistep_LSTM_run2'] = lstm_model.evaluate(multi_step_window.val)
performance['Multistep_LSTM_run2'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 77s 56ms/step - loss: 0.0105 - mean_absolute_error: 0.0710 - val_loss: 1.0433 - val_mean_absolute_error: 0.8081
Epoch 2/20
1370/1370 [==============================] - 78s 56ms/step - loss: 0.0060 - mean_absolute_error: 0.0543 - val_loss: 0.9614 - val_mean_absolute_error: 0.7729
Epoch 3/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0047 - mean_absolute_error: 0.0484 - val_loss: 0.9873 - val_mean_absolute_error: 0.8184
Epoch 4/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0055 - mean_absolute_error: 0.0520 - val_loss: 0.9548 - val_mean_absolute_error: 0.8054
Epoch 5/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0042 - mean_absolute_error: 0.0453 - val_loss: 0.8992 - val_mean_absolute_error: 0.7626
Epoch 6/20
1370/1370 [==============================] - 79s 57ms/step - loss: 0.0041 - mean_absolute_error: 0.0444 - val_loss: 0.8842 - val_mean_absolute_error:

{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831]}

In [22]:
# # Split the data
# column_indices = {name: i for i, name in enumerate(df_all_chan.columns)}
# n = len(df_all_chan)

train_df = df_all_chan[int(n*0.3):int(n*1.0)]
val_df = df_all_chan[int(n*0.1):int(n*0.3)]
test_df = df_all_chan[int(n*0.0):int(n*0.1)]

In [23]:
history = compile_and_fit(lstm_model, multi_step_window)

val_performance['Multistep_LSTM_run3'] = lstm_model.evaluate(multi_step_window.val)
performance['Multistep_LSTM_run3'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 79s 57ms/step - loss: 0.0026 - mean_absolute_error: 0.0354 - val_loss: 1.0289 - val_mean_absolute_error: 0.8188
Epoch 2/20
1370/1370 [==============================] - 80s 58ms/step - loss: 0.0024 - mean_absolute_error: 0.0344 - val_loss: 0.9420 - val_mean_absolute_error: 0.7830
Epoch 3/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0021 - mean_absolute_error: 0.0325 - val_loss: 0.8841 - val_mean_absolute_error: 0.7581
Epoch 4/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0037 - mean_absolute_error: 0.0402 - val_loss: 0.8664 - val_mean_absolute_error: 0.7514
Epoch 5/20
1370/1370 [==============================] - 78s 57ms/step - loss: 0.0019 - mean_absolute_error: 0.0310 - val_loss: 0.8481 - val_mean_absolute_error: 0.7439
Epoch 6/20
1370/1370 [==============================] - 80s 58ms/step - loss: 0.0020 - mean_absolute_error: 0.0316 - val_loss: 0.8096 - val_mean_absolute_error:

{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831],
 'Multistep_LSTM_run3': [0.22519896924495697, 0.3887842893600464]}

In [24]:
train_df = df_all_chan[int(n*0.0):int(n*0.7)]
val_df = df_all_chan[int(n*0.8):int(n*1.0)]
test_df = df_all_chan[int(n*0.7):int(n*0.8)]

history = compile_and_fit(lstm_model, multi_step_window)

val_performance['Multistep_LSTM_run4'] = lstm_model.evaluate(multi_step_window.val)
performance['Multistep_LSTM_run4'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 79s 57ms/step - loss: 0.0018 - mean_absolute_error: 0.0297 - val_loss: 0.7682 - val_mean_absolute_error: 0.7118
Epoch 2/20
1370/1370 [==============================] - 79s 57ms/step - loss: 0.0017 - mean_absolute_error: 0.0291 - val_loss: 0.8275 - val_mean_absolute_error: 0.7461
Epoch 3/20
390/390 [==============================] - 15s 21ms/step - loss: 1.0655 - mean_absolute_error: 0.8436


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831],
 'Multistep_LSTM_run3': [0.22519896924495697, 0.3887842893600464],
 'Multistep_LSTM_run4': [0.2293972223997116, 0.39348098635673523]}

In [25]:
# # Split the data
# column_indices = {name: i for i, name in enumerate(df_all_chan.columns)}
# n = len(df_all_chan)

train_df = df_all_chan[int(n*0.0):int(n*0.7)]
val_df = df_all_chan[int(n*0.8):int(n*1.0)]
test_df = df_all_chan[int(n*0.7):int(n*0.8)]

history = compile_and_fit(dnn_model, multi_step_window)

val_performance['Multistep_DNN_run2'] = dnn_model.evaluate(multi_step_window.val)
performance['Multistep_DNN_run2'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 26s 19ms/step - loss: 0.0149 - mean_absolute_error: 0.0839 - val_loss: 0.8983 - val_mean_absolute_error: 0.7937
Epoch 2/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0105 - mean_absolute_error: 0.0761 - val_loss: 0.7764 - val_mean_absolute_error: 0.7277
Epoch 3/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0088 - mean_absolute_error: 0.0698 - val_loss: 0.8201 - val_mean_absolute_error: 0.7500
Epoch 4/20
390/390 [==============================] - 12s 9ms/step - loss: 0.8236 - mean_absolute_error: 0.7638


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831],
 'Multistep_LSTM_run3': [0.22519896924495697, 0.3887842893600464],
 'Multistep_LSTM_run4': [0.2293972223997116, 0.39348098635673523],
 'Multistep_DNN_run2': [0.23267138004302979, 0.3909773826599121]}

In [26]:
train_df = df_all_chan[int(n*0.3):int(n*1.0)]
val_df = df_all_chan[int(n*0.1):int(n*0.3)]
test_df = df_all_chan[int(n*0.0):int(n*0.1)]

history = compile_and_fit(dnn_model, multi_step_window)

val_performance['Multistep_DNN_run3'] = dnn_model.evaluate(multi_step_window.val)
performance['Multistep_DNN_run3'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0080 - mean_absolute_error: 0.0672 - val_loss: 0.7200 - val_mean_absolute_error: 0.6977
Epoch 2/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0074 - mean_absolute_error: 0.0648 - val_loss: 0.7158 - val_mean_absolute_error: 0.6967
Epoch 3/20
1370/1370 [==============================] - 24s 18ms/step - loss: 0.0071 - mean_absolute_error: 0.0625 - val_loss: 0.8135 - val_mean_absolute_error: 0.7553
Epoch 4/20
390/390 [==============================] - 11s 9ms/step - loss: 0.8378 - mean_absolute_error: 0.7725


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831],
 'Multistep_LSTM_run3': [0.22519896924495697, 0.3887842893600464],
 'Multistep_LSTM_run4': [0.2293972223997116, 0.39348098635673523],
 'Multistep_DNN_run2': [0.23267138004302979, 0.3909773826599121],
 'Multistep_DNN_run3': [0.238728329539299, 0.4001675248146057]}

In [27]:
train_df = df_all_chan[int(n*0.0):int(n*0.7)]
val_df = df_all_chan[int(n*0.8):int(n*1.0)]
test_df = df_all_chan[int(n*0.7):int(n*0.8)]

history = compile_and_fit(dnn_model, multi_step_window)

val_performance['Multistep_DNN_run4'] = dnn_model.evaluate(multi_step_window.val)
performance['Multistep_DNN_run4'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1370/1370 [==============================] - 26s 18ms/step - loss: 0.0068 - mean_absolute_error: 0.0603 - val_loss: 0.9054 - val_mean_absolute_error: 0.8066
Epoch 2/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0062 - mean_absolute_error: 0.0591 - val_loss: 0.8583 - val_mean_absolute_error: 0.7790
Epoch 3/20
1370/1370 [==============================] - 25s 18ms/step - loss: 0.0063 - mean_absolute_error: 0.0591 - val_loss: 0.7867 - val_mean_absolute_error: 0.7373
Epoch 4/20
1370/1370 [==============================] - 26s 19ms/step - loss: 0.0058 - mean_absolute_error: 0.0570 - val_loss: 0.9322 - val_mean_absolute_error: 0.8241
Epoch 5/20
390/390 [==============================] - 12s 10ms/step - loss: 0.8484 - mean_absolute_error: 0.7709


{'Linear': [0.5327640175819397, 0.5664042830467224],
 'Multistep_Linear': [0.4156869649887085, 0.49181458353996277],
 'Multistep_LSTM': [0.287476509809494, 0.41880205273628235],
 'Multistep_DNN': [0.2558583915233612, 0.40451568365097046],
 'Multistep_LSTM_run2': [0.21725155413150787, 0.3834638297557831],
 'Multistep_LSTM_run3': [0.22519896924495697, 0.3887842893600464],
 'Multistep_LSTM_run4': [0.2293972223997116, 0.39348098635673523],
 'Multistep_DNN_run2': [0.23267138004302979, 0.3909773826599121],
 'Multistep_DNN_run3': [0.238728329539299, 0.4001675248146057],
 'Multistep_DNN_run4': [0.22017206251621246, 0.3799917697906494]}

In [32]:
print(train_std['Discharge'], train_mean['Discharge'])

3.149408744056611 11.432633314265177


In [33]:
(0.21725155413150787 * train_std['Discharge']) + train_mean['Discharge']

12.116847258506835

In [34]:
(0.21725155413150787 * train_std['Discharge'])

0.684213944241659